<a href="https://colab.research.google.com/github/laurenz-coac/mless/blob/main/homework2/5_LSTM_ozone_future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Approaches:

If we want to incorporate future temperature data to predict current ozone measurements, we have two approaches that do not require architectural changes:

1. **Shifting the temperature measurements backwards:** Say our context window starts at $t=0$ and goes until timepoint $t=w_c$. Our prediciton window then, of course, starts at $t=w_c +1$ and goes until $t=w_c + w_p$, where $w_p$ is the length of the prediciton window. In this case (if we have two input variables), the input dimensions to our model are $(N, w_c, 2)$. To include future data for the temperature variable, we can shift the temperature data by $k$ timepoints, s.t. the first measurement included in the context window would be from timepoint $t=k$, while the last timepoint would be at $t=w_c + k$. This way, the model can see $k$ steps into the future, at the cost of losing the first $k$ measurements for the temperature data.

2. **Appending and Padding:** If we do not want to lose the information at the beginning of the temperature sequence, we can also append the whole future window for this variable to our context window. Then the data would be of dimensionality $(N, w_c + w_p, 2)$, where the last $w_p$ indices for ozone would be some padding value (probably best to pick an out-of-distribution value). If the model struggles with the large sequence of pads, the task could also be reformulated to a single-step prediciton task.

(**A probably better apporach:** If we can make architectural changes, it would probably be a good idea to revert to an encoder-decoder architecture, where the _past_ data is encoded together, and future temperature data is only given to the decoder while predicting ozone values. In this way, we have an explicit distinction between past and future values, which can probably be learned more efficiently by the model.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, LSTM, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import os

context_window = 336
prediction_horizon = 96
variable_column = ["temp", "o3"] # define the variables wanted for training

In [2]:
# Function to evaluate model performance
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"RMSE: {rmse:.4f}")
    return rmse

# Loading multi-variable data-sequence

Here the same as in the original ozone-prediction notebook is done.

In [3]:
from re import X
import pickle

# Load the prepared multi-variable data
with open("X_train.pkl", "rb") as f:
    X_train_full = pickle.load(f)

with open("X_test.pkl", "rb") as f:
    X_test_full = pickle.load(f)

with open("y_train.pkl", "rb") as f:
    y_train_full = pickle.load(f)

with open("y_test.pkl", "rb") as f:
    y_test_full = pickle.load(f)

print(f"X_train_full shape: {X_train_full.shape}, y_train_full shape: {y_train_full.shape}")
print(f"X_test_full shape: {X_test_full.shape}, y_test_full shape: {y_test_full.shape}")

## Else if using local files:
dataframe = pd.read_csv("normalized_data.csv")
scaler_stats = {col: {'mean': dataframe[col].mean(), 'std': dataframe[col].std()} for col in variable_column}


# the station code is the first variable column, hence select only the last two
X_train = X_train_full[:,:,1:].copy()
X_test = X_test_full[:,:,1:].copy()

# for the label, we only want the ozone data, which is the second column

temp_y_train = y_train_full[:,:,1].copy()  # temperature data for training
temp_y_test = y_test_full[:,:,1].copy()    # temperature data for testing

y_train = y_train_full[:,:,2].copy()
y_test = y_test_full[:,:,2].copy()

X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
temp_y_train = np.array(temp_y_train, dtype=np.float32)
temp_y_test = np.array(temp_y_test, dtype=np.float32)

# verify the shapes of the data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train_full shape: (21493, 336, 3), y_train_full shape: (21493, 96, 3)
X_test_full shape: (9212, 336, 3), y_test_full shape: (9212, 96, 3)
X_train shape: (21493, 336, 2), y_train shape: (21493, 96)
X_test shape: (9212, 336, 2), y_test shape: (9212, 96)


## Define Training Function

The model is trained in the same way for both approaches, only the data preparation is different. The only thing we need to account for is the context-window length.

In [9]:
def train_lstm_model(X_train, y_train, context_window):

    # Tunable LSTM parameters
    lstm_units = 50
    lstm_epochs = 5
    lstm_batch_size = 16
    lstm_optim = 'adam'
    lstm_loss = 'mse'

    checkpoint_dir = "./checkpoint/"
    os.makedirs(checkpoint_dir, exist_ok=True)

    checkpoint_path = os.path.join(checkpoint_dir, f"lstm_multivar.h5")

    ## Ignore user warning on keras as the choice for this exercise is to use h5.
    print(f"Training new model for variables {variable_column}")

    # the only change needed to allow for multiple input variables is to change the input shape of the LSTM layer
    # to match the number of variables in the input data
    lstm_model = Sequential([
        LSTM(lstm_units, return_sequences=True, input_shape=(context_window, len(variable_column))), # change to allow mulitple input variables
        LSTM(lstm_units, return_sequences=False),
        Dense(prediction_horizon)
    ])

    lstm_model.compile(optimizer="adam", loss="mse")

    checkpoint_callback = ModelCheckpoint(
        checkpoint_path, monitor="val_loss", save_best_only=True, verbose=1
    )

    training = lstm_model.fit(
        X_train,
        y_train,
        epochs=lstm_epochs, batch_size=lstm_batch_size,
        validation_split=0.2, verbose=1,
        callbacks=[checkpoint_callback]
    )

    training_history = training.history

    return lstm_model

In [10]:
def get_ozone_predictions(model, X_test, y_test):
    """
    Get ozone predictions from the trained model.
    """
    lstm_pred = model.predict(X_test)
    rmse = evaluate_model(y_test, lstm_pred)

    return lstm_pred

In [11]:
def save_predictions(lstm_pred, filename):

  X_first_idx = np.flatnonzero(X_test_full[:, 0, 0] == 'DENW094')[0]
  context=X_test_full[X_first_idx, :, 2] # for comparability, stick with given code, even though not necessary
  # First sample of DENW094 station to compare with PatchTST

  actual_future = y_test[X_first_idx, :]
  predicted_future = lstm_pred[X_first_idx, :] #actual and pred denormalized in prev cell

  # Inverse scale
  predicted_future = predicted_future * scaler_stats[variable_column[1]]['std'] + scaler_stats[variable_column[1]]['mean']

  #write forecast values to csv
  forecast_df = pd.DataFrame({
      'timepoints': range(context_window, context_window + prediction_horizon),
      'forecast': predicted_future
  })

  forecast_df.to_csv(filename, index=False)

# Approach 1

## Preparing the data

In [12]:
k = 96 # shift the temp data by k hours to use future temperature data
# play around with k to see impact

temp_train = X_train[:, k:, 0].copy() # take the temperature data from the training set
temp_future = temp_y_train[:, :k].copy() # take the temperature data from the future set

In [13]:
temp_train.shape, temp_future.shape

((21493, 240), (21493, 96))

In [14]:
concat_temp = np.concatenate((temp_train, temp_future), axis=1) # concatenate the two temperature data sets

# replace temperature data in the training set with the concatenated data
X_train1 = X_train.copy()  # create a copy of the training data to avoid modifying the original
X_train1[:, :, 0] = concat_temp

In [15]:
# train the LSTM model with the modified training data
lstm_model = train_lstm_model(X_train1, y_train, context_window)

Training new model for variables ['temp', 'o3']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1073/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.6623
Epoch 1: val_loss improved from inf to 0.62365, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 30s 25ms/step - loss: 0.6620 - val_loss: 0.6237
Epoch 2/5
1073/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4449
Epoch 2: val_loss improved from 0.62365 to 0.54300, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 26s 24ms/step - loss: 0.4449 - val_loss: 0.5430
Epoch 3/5
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4223
Epoch 3: val_loss improved from 0.54300 to 0.49731, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 26s 24ms/step - loss: 0.4223 - val_loss: 0.4973
Epoch 4/5
1073/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3475
Epoch 4: val_loss did not improve from 0.49731
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - loss: 0.3475 - val_loss: 0.5217
Epoch 5/5
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3484
Epoch 5: val_loss did not improve from 0.49731
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 43s 27ms/step - loss: 0.3484 - val_loss: 0.5113


In [16]:
# prepare test data in the same way

temp_test = X_test[:, k:, 0].copy() # take the temperature data from the test set
temp_future = temp_y_test[:, :k].copy() # take the temperature data from the future set

concat_temp = np.concatenate((temp_test, temp_future), axis=1) # concatenate the two temperature data sets

# replace temperature data in the test set with the concatenated data
X_test1 = X_test.copy()  # create a copy of the test data to avoid modifying the original
X_test1[:, :, 0] = concat_temp

In [17]:
lstm_pred = get_ozone_predictions(lstm_model, X_test1, y_test)

288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
RMSE: 0.6740


In [21]:
save_predictions(lstm_pred, "LSTM_futureA1_forecast_k96.csv")

# Approach 2

## Preparing the data

In [19]:
y_train_vars = y_train_full[:, :, 1:]

X_train2 = np.concatenate((X_train, y_train_vars), axis=1)  # concatenate the training data with the test data

# mask out the ozone data in the training set
X_train2[:, context_window:, 1] = -99
X_train2 = np.array(X_train2, dtype=np.float32)

In [22]:
X_train2.shape

(21493, 432, 2)

In [23]:
# train the LSTM model with the modified training data
lstm_model = train_lstm_model(X_train2, y_train, X_train2.shape[1])

Training new model for variables ['temp', 'o3']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1074/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.7411
Epoch 1: val_loss improved from inf to 0.56473, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 36s 31ms/step - loss: 0.7408 - val_loss: 0.5647
Epoch 2/5
1073/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.4875
Epoch 2: val_loss improved from 0.56473 to 0.55644, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 33s 30ms/step - loss: 0.4875 - val_loss: 0.5564
Epoch 3/5
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.4385
Epoch 3: val_loss improved from 0.55644 to 0.53643, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 31s 29ms/step - loss: 0.4385 - val_loss: 0.5364
Epoch 4/5
1074/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.4040
Epoch 4: val_loss improved from 0.53643 to 0.51672, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 43s 30ms/step - loss: 0.4040 - val_loss: 0.5167
Epoch 5/5
1075/1075 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.3753
Epoch 5: val_loss improved from 0.51672 to 0.50961, saving model to ./checkpoint/lstm_multivar.h5


1075/1075 ━━━━━━━━━━━━━━━━━━━━ 33s 31ms/step - loss: 0.3753 - val_loss: 0.5096


In [24]:
# prepare the test data in the same way

y_test_vars = y_test_full[:, :, 1:]

X_test2 = np.concatenate((X_test, y_test_vars), axis=1)  # concatenate
X_test2[:, context_window:, 1] = -99
X_test2 = np.array(X_test2, dtype=np.float32)

lstm_pred = get_ozone_predictions(lstm_model, X_test2, y_test)

288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
RMSE: 0.6370


In [32]:
save_predictions(lstm_pred, "LSTM_futureA2_forecast.csv")